[参考](https://www.kaggle.com/startupsci/titanic-data-science-solutions).

In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [2]:
combine = pd.concat([train_df, test_df], sort=False)
combine.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
combine.drop(['Ticket', 'Cabin'], axis=1, inplace=True)

In [4]:
# The expand=False flag returns a DataFrame.
combine['Title'] = combine['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
# pd.crosstab(combine['Title'], combine['Sex'])

In [5]:
combine['Title'] = combine['Title'].replace([
    'Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir',
    'Jonkheer', 'Dona'
], 'Rare')

combine['Title'] = combine['Title'].replace('Mlle', 'Miss')
combine['Title'] = combine['Title'].replace('Ms', 'Miss')
combine['Title'] = combine['Title'].replace('Mme', 'Mrs')

# combine[['Title', 'Survived']].groupby(['Title'], as_index=False).mean().sort_values(by='Survived')

In [6]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

combine['Title'] = combine['Title'].map(title_mapping)
combine['Title'].value_counts(dropna=None)

1    757
2    264
3    198
4     61
5     29
Name: Title, dtype: int64

In [7]:
combine.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,3
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,2
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,3
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1


In [8]:
combine.drop(['Name', 'PassengerId', 'Survived'], axis=1, inplace=True)

In [9]:
combine['Sex'] = combine['Sex'].map({'female': 0, 'male': 1})
combine['Age'].fillna(combine['Age'].median(), inplace=True)

分组估计年龄

用 transform 函数试试看。

```python
guess_ages = np.zeros((2,3))
guess_ages

for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()
```

In [10]:
combine['Age'].min(), combine['Age'].max()

(0.17, 80.0)

In [11]:
out, bins = pd.cut(combine['Age'], 5, retbins=True)
bins

array([ 0.09017, 16.136  , 32.102  , 48.068  , 64.034  , 80.     ])

In [12]:
def age_cut(age):
    if age <= 16:
        return 0
    if age <= 32:
        return 1
    if age <= 48:
        return 2
    if age <= 64:
        return 3
    return 4

In [13]:
combine['Age'] = combine['Age'].map(age_cut)    

In [14]:
combine['FamilySize'] = combine['SibSp'] + combine['Parch'] + 1
combine['IsAlone'] = (combine['FamilySize'] == 1).astype(int)
# combine[(combine['Parch'] == 0) & (combine['SibSp'] == 0)]
combine.drop(['Parch', 'SibSp', 'FamilySize'], axis=1, inplace=True)

In [15]:
combine['Age*Class'] = combine.Age * combine.Pclass

In [16]:
freq_port = combine.Embarked.dropna().mode()[0]
freq_port

'S'

In [17]:
combine['Embarked'].fillna(freq_port, inplace=True)

In [18]:
combine['Embarked'] = combine['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

In [19]:
combine['Fare'].fillna(combine['Fare'].dropna().median(), inplace=True)

In [20]:
out, bins = pd.qcut(combine['Fare'], 4, retbins=True)
bins

array([  0.    ,   7.8958,  14.4542,  31.275 , 512.3292])

In [21]:
def fare_cut(age):
    if age <=  7.8958:
        return 0
    if age <= 14.4542:
        return 1
    if age <= 31.275:
        return 2
    return 3

In [22]:
combine['Fare'] = combine['Fare'].map(fare_cut)

In [23]:
X = combine.iloc[:train_df.shape[0]]
y = train_df["Survived"]
X_test = combine.iloc[train_df.shape[0]:]

In [40]:
X.head()

,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,3,1,1,0,0,1,0,3
1,1,0,2,3,1,3,0,2
2,3,0,1,1,0,2,1,3
3,1,0,2,3,0,3,0,2
4,3,1,2,1,0,1,1,6


## 下面开始建模

In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression


logreg = LogisticRegression(solver='newton-cg')
logreg_scores = cross_val_score(logreg, X, y, cv=10)
logreg_scores_mean = logreg_scores.mean()

In [25]:
logreg.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [26]:
def func_abs(x):
    return abs(x)


def sign(x):
    return int(x >= 0)


pd.Series(
    logreg.coef_[0], index=X.columns).transform([func_abs, sign]).sort_values(
        by='func_abs', ascending=False)

,func_abs,sign
Sex,2.204133,0
Pclass,1.001067,0
Title,0.399469,1
Age,0.291285,0
Embarked,0.281860,1
IsAlone,0.241214,1
Age*Class,0.102954,0
Fare,0.013072,1


In [27]:
from sklearn.svm import SVC

svc = SVC()

svc_scores = cross_val_score(svc, X, y, cv=10)
svc_scores_mean = svc_scores.mean()
svc_scores_mean

/Users/liwei/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/svm/base

0.8125922142776074

In [28]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn_scores = cross_val_score(knn, X, y, cv=10)
knn_scores_mean = knn_scores.mean()
knn_scores_mean

0.7959130632164341

In [29]:
from sklearn.naive_bayes import GaussianNB

gaussian = GaussianNB()

gaussian_scores = cross_val_score(gaussian, X, y, cv=10)
gaussian_scores_mean = gaussian_scores.mean()

In [30]:
from sklearn.linear_model import Perceptron

perceptron = Perceptron()

perceptron_scores = cross_val_score(perceptron, X, y, cv=10)
perceptron_scores_mean = perceptron_scores.mean()

/Users/liwei/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max

In [32]:
from sklearn.svm import LinearSVC

linear_svc = LinearSVC()

linear_svc_scores = cross_val_score(linear_svc, X, y, cv=10)
linear_svc_scores_mean = linear_svc_scores.mean()

/Users/liwei/.local/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the

In [33]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()

sgd_scores = cross_val_score(sgd, X, y, cv=10)
sgd_scores_mean = sgd_scores.mean()

/Users/liwei/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they defau

In [34]:
from sklearn.tree import DecisionTreeClassifier


decision_tree = DecisionTreeClassifier()

decision_tree_scores = cross_val_score(decision_tree, X, y, cv=10)
decision_tree_scores_mean = decision_tree_scores.mean()

In [35]:
from sklearn.ensemble import RandomForestClassifier


random_forest = RandomForestClassifier(n_estimators=100)

random_forest_scores = cross_val_score(random_forest, X, y, cv=10)
random_forest_scores_mean = random_forest_scores.mean()

In [36]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [svc_scores_mean, knn_scores_mean, logreg_scores_mean, 
              random_forest_scores_mean, gaussian_scores_mean, perceptron_scores_mean, 
              sgd_scores_mean, linear_svc_scores_mean, decision_tree_scores_mean]})
models.sort_values(by='Score', ascending=False)

,Model,Score
8,Decision Tree,0.812667
0,Support Vector Machines,0.812592
3,Random Forest,0.810420
2,Logistic Regression,0.803565
1,KNN,0.795913
7,Linear SVC,0.791193
4,Naive Bayes,0.766648
6,Stochastic Gradient Decent,0.739620
5,Perceptron,0.663250


In [37]:
random_forest.fit(X,y)
y_pred =random_forest.predict(X_test)

In [38]:
from xgboost import XGBClassifier

xgbc = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
xgbc.fit(X, y)
y_pred = xgbc.predict(X_test)
# 0.80861

In [39]:
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": y_pred
})
submission.to_csv('../output/submission_xgbc.csv', index=False)